In [50]:
import requests
import json
import pandas as pd
import os

#external modules
import api_call_helpers as ach
import bigquery_functions as pp
# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# set up params
USERNAME = 'tanas0rn'
API_KEY = 'RGAPI-21e3113c-6f6c-4b33-8cd0-721e8bb19c49'
ORIGIN = "https://developer.riotgames.com"
ACCEPT_CHARSET = "application/x-www-form-urlencoded; charset=UTF-8"
ACCEPT_LANGUAGE = "de-DE,de;q=0.9,en-US;q=0.8,en;q=0.7,en-GB;q=0.6"
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"

headers = {
    "User-Agent": USER_AGENT,
    "Accept-Language": ACCEPT_LANGUAGE,
    "Accept-Charset": ACCEPT_CHARSET,
    "Origin": ORIGIN,
    "X-Riot-Token": API_KEY
}

In [3]:
# connect to BigQuery 

In [4]:
# get summoner info (account ID etc)
encrypted_account_id = ach.get_encrypted_accid(USERNAME,headers)

 Request Successful


In [5]:
# get match list for summoner
match_list = ach.get_match_list_by_accid(encrypted_account_id,headers)

 Request Successful
 Request Successful
 Request Successful
Total 255 Matches Retrieved


In [6]:
# get all of the match IDs
match_ids = [item['gameId'] for item in match_list]
match_id = match_ids[0]

In [7]:
# get match stats
matchstats = []
for match_id in match_ids[:5]:
    stats = ach.get_match_stats(USERNAME,match_id,headers)
    
    matchstats.append(stats)

 Request Successful
 Request Successful
 Request Successful
 Request Successful
 Request Successful


# Data preprocessing (to tuples)

df = pd.DataFrame(matchstats)
for col in df.columns:
    print(col)

In [93]:
keys_to_remove = (
    'item0',
    'item1',
    'item2',
    'item3',
    'item4',
    'item5',
    'item6',
    'playerScore0',
    'playerScore1',
    'playerScore2',
    'playerScore3',
    'playerScore4',
    'playerScore5',
    'playerScore6',
    'playerScore7',
    'playerScore8',
    'playerScore9',
    'perk0',
    'perk0Var1',
    'perk0Var2',
    'perk0Var3',
    'perk1',
    'perk1Var1',
    'perk1Var2',
    'perk1Var3',
    'perk2',
    'perk2Var1',
    'perk2Var2',
    'perk2Var3',
    'perk3',
    'perk3Var1',
    'perk3Var2',
    'perk3Var3',
    'perk4',
    'perk4Var1',
    'perk4Var2',
    'perk4Var3',
    'perk5',
    'perk5Var1',
    'perk5Var2',
    'perk5Var3',
    'perkPrimaryStyle',
    'perkSubStyle',
    'statPerk0',
    'statPerk1',
    'statPerk2',
    'firstInhibitorKill', 
    'firstInhibitorAssist'
)

In [94]:
for match in matchstats:
    pp.entries_to_remove(keys_to_remove,match)

In [95]:
#convert to tuples
matchstats_tuples = [tuple(d.values()) for d in matchstats]

# Insert data into bigquery table

In [99]:
from google.cloud import bigquery
from google.oauth2 import service_account

# connect to bigquery

path = os.getcwd()
secrets_filename = 'bigquery_auth.json'

CREDENTIALS = service_account.Credentials.from_service_account_file(
    f'{path}/{secrets_filename}')
PROJECT_ID = 'riotdashboard-283414'
DATASET_ID = 'RIOT_DWH'
TABLE_ID = 'MATCH_STATISTICS'

client = bigquery.Client(credentials= credentials,project=project_id)

In [100]:
# get list of all existing matches/players
query = """
SELECT matchID, summonername
FROM RIOT_DWH.MATCH_STATISTICS;
"""

query_job = client.query(query)
results = query_job.result() 

In [101]:
results

In [109]:
# Insert rows to table
pp.insert_rows_to_tables(credentials = CREDENTIALS,
                          project_id = PROJECT_ID,
                          dataset_id = DATASET_ID,
                          table_id = TABLE_ID,
                          insert_rows = matchstats_tuples[0])